In [32]:
import requests as re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import folium
from folium.plugins import HeatMap
import os
import calendar

In [33]:
url = "http://universities.hipolabs.com/search?country=United+States"
r = re.get(url)
r_json = r.json()
df = pd.json_normalize(data = r_json)
df

,state-province,domains,country,web_pages,name,alpha_two_code
0,None,[marywood.edu],United States,[http://www.marywood.edu],Marywood University,US
1,None,[lindenwood.edu],United States,[http://www.lindenwood.edu/],Lindenwood University,US
2,None,[sullivan.edu],United States,[https://sullivan.edu/],Sullivan University,US
3,None,[fscj.edu],United States,[https://www.fscj.edu/],Florida State College at Jacksonville,US
4,None,[xavier.edu],United States,[https://www.xavier.edu/],Xavier University,US
...,...,...,...,...,...,...
4557,None,[vul.edu],United States,[https://www.vul.edu/],Virginia University of Lynchburg,US
4558,None,[voorhees.edu],United States,[https://www.voorhees.edu/],Voorhees University,US
4559,None,[wvstate.edu],United States,[https://www.wvstateu.edu/],West Virginia State University,US
4560,None,[wileyc.edu],United States,[https://www.wileyc.edu/],Wiley College,US


In [34]:
print(df.shape)
df.head()

(4562, 6)


,state-province,domains,country,web_pages,name,alpha_two_code
0,None,[marywood.edu],United States,[http://www.marywood.edu],Marywood University,US
1,None,[lindenwood.edu],United States,[http://www.lindenwood.edu/],Lindenwood University,US
2,None,[sullivan.edu],United States,[https://sullivan.edu/],Sullivan University,US
3,None,[fscj.edu],United States,[https://www.fscj.edu/],Florida State College at Jacksonville,US
4,None,[xavier.edu],United States,[https://www.xavier.edu/],Xavier University,US


In [35]:
df.columns

Index(['state-province', 'domains', 'country', 'web_pages', 'name',
       'alpha_two_code'],
      dtype='object')

In [101]:
#Column rename
fixed_columns = {
    'state-province':'State',
    'domains':'Domain',
    'country':'Country_name',
    'web_pages':'web_page_name',
    'name':'university_name',
    'alpha_two_code':'alpha_code',
    
}

df.rename(columns=fixed_columns,inplace=True)
df.columns

Index(['Domain', 'Country_name', 'alpha_code', 'web_page_name', 'State',
       'university_name'],
      dtype='object')

In [44]:
df.university_name.unique()

array(['Marywood University', 'Lindenwood University',
       'Sullivan University', ..., 'West Virginia State University',
       'Wiley College', 'Winston-Salem State University'], dtype=object)

In [43]:
df[df.alpha_code.str.contains('null')].head()

,State,Domain,Country_name,web_page_name,university_name,alpha_code


In [39]:
df.alpha_code.replace({' null':''}, regex=True,inplace=True)

In [41]:
df[df.alpha_code.str.contains('null')].head()

,State,Domain,Country_name,web_page_name,university_name,alpha_code


In [45]:
#Count before removing duplicates

df.shape

(4562, 6)

In [47]:
#Removing Duplicates

df = df[['Domain','Country_name','alpha_code','web_page_name','State','university_name']].loc[df[['Domain','Country_name','alpha_code','web_page_name','State','university_name']].astype(str).drop_duplicates().index]


In [48]:
#Count After removing duplicates

df.shape

(2281, 6)

In [49]:
df.head()

,Domain,Country_name,alpha_code,web_page_name,State,university_name
0,[marywood.edu],United States,US,[http://www.marywood.edu],None,Marywood University
1,[lindenwood.edu],United States,US,[http://www.lindenwood.edu/],None,Lindenwood University
2,[sullivan.edu],United States,US,[https://sullivan.edu/],None,Sullivan University
3,[fscj.edu],United States,US,[https://www.fscj.edu/],None,Florida State College at Jacksonville
4,[xavier.edu],United States,US,[https://www.xavier.edu/],None,Xavier University


In [58]:
df[df['State'] == 'nan']

,Domain,Country_name,alpha_code,web_page_name,State,university_name


In [94]:
#Data before replacing value with null 
df[df['State'] == 'Washington']

,Domain,Country_name,alpha_code,web_page_name,State,university_name
1170,[wallawalla.edu],United States,,[http://www.wallawalla.edu/],Washington,Walla Walla University
2211,[bellevuecollege.edu],United States,US,[https://bellevuecollege.edu],Washington,Bellevue College


In [95]:
#replace value with null

df.at[1170,'alpha_code']='null'

In [96]:
#Data before replacing value with null 

df[df['State'] == 'Washington']

,Domain,Country_name,alpha_code,web_page_name,State,university_name
1170,[wallawalla.edu],United States,null,[http://www.wallawalla.edu/],Washington,Walla Walla University
2211,[bellevuecollege.edu],United States,US,[https://bellevuecollege.edu],Washington,Bellevue College


In [97]:
#identify the data with null values in alpha code column

df[df.alpha_code.str.contains('null')].head()

,Domain,Country_name,alpha_code,web_page_name,State,university_name
1170,[wallawalla.edu],United States,null,[http://www.wallawalla.edu/],Washington,Walla Walla University


In [98]:
#replace the data with null values in alpha code column with value US

df.alpha_code.replace({'null':'US'}, regex=True,inplace=True)

In [99]:
df[df.alpha_code.str.contains('null')].head()

,Domain,Country_name,alpha_code,web_page_name,State,university_name


In [100]:
df[df['State'] == 'Washington']

,Domain,Country_name,alpha_code,web_page_name,State,university_name
1170,[wallawalla.edu],United States,US,[http://www.wallawalla.edu/],Washington,Walla Walla University
2211,[bellevuecollege.edu],United States,US,[https://bellevuecollege.edu],Washington,Bellevue College
